In [37]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
print("tensorflow version {}".format(tf.VERSION)) # make sure its tensorflow 1.*.*

tensorflow version 1.12.0


In [38]:
#c=a+b
#d=c*a
g0=tf.Graph()
with g0.as_default(): # context manager
    a = tf.constant([[1],[2]], dtype=tf.float32,name='a_constant')
    b = tf.constant([[2],[3]], dtype=tf.float32,name='b_constant')
    c=tf.add(a,b,name='add')  # c = a + b
    d=c * a #d = c * a
print(a)
print(b)
print(c)
print(d)
#building the graph (the actual computation does not happen here, though its tempting to think of that)
#build the engine

Tensor("a_constant:0", shape=(2, 1), dtype=float32)
Tensor("b_constant:0", shape=(2, 1), dtype=float32)
Tensor("add:0", shape=(2, 1), dtype=float32)
Tensor("mul:0", shape=(2, 1), dtype=float32)


In [39]:
#computation occurs
#start the engine
sess0=tf.Session(graph=g0)
check = ['a','c','d']
result=sess0.run([a,c,d])
for i, re in zip(check,result):
    print('value of {} =\n {}'.format(i,re))

#visualize the graph
writer = tf.summary.FileWriter('.',sess0.graph)
writer.flush()

sess0.close()

value of a =
 [[1.]
 [2.]]
value of c =
 [[3.]
 [5.]]
value of d =
 [[ 3.]
 [10.]]


In [40]:
!pwd

/Users/haobei/Projects/Tutorial/L3


In [41]:
#quiz 1
#g0_q=tf.Graph()
#c = a @ b
#a.shape=(4,2), b.shape=(2,3)
g0_q=tf.Graph()
with g0_q.as_default():
    a = tf.constant(tf.ones(4,2),dtype=tf.float32)
    b = tf.constant(tf.ones(2,3),dtype=tf.float32)
    c = a @ b
sess0q=g

TypeError: Expected float32, got list containing Tensors of type '_Message' instead.

In [42]:
#tensor math matrix
#with "defaultGraph".as_default():
g1=tf.Graph()
a0 = tf.constant(2)
with g1.as_default():
    a = tf.random_normal([2,3], dtype=tf.float32,name='a_random')
    b = tf.constant([[1,1],[1,1],[1,1]], dtype=tf.float32,name='b')
    op = tf.print("value of a",a,name='print')
    var_x = tf.get_variable("var_x", [2,2], dtype=tf.float32, trainable=True,
                            initializer=tf.glorot_normal_initializer)
    c = tf.matmul(a,b,name='matmul') # a @ b
    d = c + var_x #tf.add
print(a0,a0.graph)
print(a,a.graph)
print(b,b.graph)
print(op,op.graph)
print(var_x,var_x.graph)
print(c,c.graph)
print(d,d.graph)

Tensor("Const_1:0", shape=(), dtype=int32) <tensorflow.python.framework.ops.Graph object at 0xb3b4e9b00>
Tensor("a_random:0", shape=(2, 3), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f0de080>
Tensor("b:0", shape=(3, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f0de080>
name: "print"
op: "PrintV2"
input: "print_format"
attr {
  key: "output_stream"
  value {
    s: "stderr"
  }
}
<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref> <tensorflow.python.framework.ops.Graph object at 0xb3f0de080>
Tensor("matmul:0", shape=(2, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f0de080>
Tensor("add:0", shape=(2, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f0de080>


In [43]:
#engine switch
sess1=tf.Session(graph=g1)

In [44]:
print('print',sess1.run(op))
print('a',sess1.run(a)) #print(a.eval(session=sess))
print('matmul',sess1.run('matmul'))
print('c',sess1.run(c)) #c
print('a,c',sess1.run([a,c,a]))
# print('initilizer',sess1.run(var_x.initializer))
with g1.as_default():
    sess1.run(tf.global_variables_initializer())
print('d',sess1.run(d))
print('var_x',sess1.run(var_x))
print('d',sess1.run(d))
print('assign',sess1.run(var_x.assign_add([[1,0],[1,1]])))
print('var_x,d',sess1.run([var_x,d]))

print None
a [[ 1.8965936   0.67290306  0.38004753]
 [ 0.6080279  -1.3229957   0.53101563]]
matmul None
c [[-2.360977 -2.360977]
 [-0.931275 -0.931275]]
a,c [array([[0.03357645, 0.672558  , 0.56179285],
       [1.1359289 , 0.03175025, 0.53392935]], dtype=float32), array([[1.2679273, 1.2679273],
       [1.7016084, 1.7016084]], dtype=float32), array([[0.03357645, 0.672558  , 0.56179285],
       [1.1359289 , 0.03175025, 0.53392935]], dtype=float32)]
d [[ 0.84133595 -0.20170203]
 [ 0.04592577  0.09867543]]
var_x [[ 0.78981847 -0.25321952]
 [ 0.30114126  0.35389093]]
d [[-3.6264184  -4.6694565 ]
 [-0.99057496 -0.9378253 ]]
assign [[ 1.7898185  -0.25321952]
 [ 1.3011413   1.3538909 ]]
var_x,d [array([[ 1.7898185 , -0.25321952],
       [ 1.3011413 ,  1.3538909 ]], dtype=float32), array([[ 1.8856635 , -0.15737459],
       [-0.6366663 , -0.58391666]], dtype=float32)]


In [45]:
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess1.graph)
writer.flush()
tf.reset_default_graph()
v = tf.get_variable("d",shape=[2,3],dtype=tf.float32,initializer=tf.glorot_normal_initializer)

In [46]:
#facilitate updates of weights of a neural network
with g1.as_default():
    sess1.run(tf.global_variables_initializer())
    print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)) 
    print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref>]
[<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref>]
[<tf.Variable 'd:0' shape=(2, 3) dtype=float32_ref>]


In [47]:
sess1.close()

In [48]:
#quiz 2
#g1_q=tf.Graph()
#d = tf.reduce_sum(var_w * ((L @ x)^2)) + var_b
# var_w.shape=(2,) L.shape=(2,3) x.shape=(3,1) var_b.shape=() initializer: tf.random_normal_initializer
g1_q=tf.Graph()
with g1_q.as_default():
    x = tf.constant([[0],[1],[2]],dtype=tf.float32)
    var_w = tf.get_variable('var_w',shape=(2,1),dtype=tf.float32,initializer=tf.ones_initializer)
    L = tf.constant([[0,1,2],[2,3,4]],name='L',dtype=tf.float32)
    var_b = tf.get_variable('var_b',shape=(),dtype=tf.float32,initializer=tf.ones_initializer)
    d=tf.reduce_sum(var_w * tf.square(L @ x))+var_b

In [49]:
sess_g1_q = tf.Session(graph=g1_q)
# sess_g1_q.run([var_w.initializer,var_b.initializer])
with g1_q.as_default():
    sess_g1_q.run(tf.global_variables_initializer())
print(sess_g1_q.run(d))
writer = tf.summary.FileWriter('.',sess_g1_q.graph)
writer.flush()

147.0


In [66]:
x = np.array([[1], [2], [3], [4]],dtype=np.float32)
y = np.array([[4], [3], [2], [1]],dtype=np.float32)

In [67]:
# build default graph
tf.reset_default_graph()
# mini-batch 
x_p = tf.placeholder(dtype=tf.float32,shape=(None,1),name="x_placeholder") # shape is determained at runtime
y_p = tf.placeholder(dtype=tf.float32,shape=(None,1),name="y_placeholder")

W = tf.get_variable("weight",shape=(),dtype=tf.float32,
                    initializer=tf.ones_initializer,
                    trainable=True)
b = tf.get_variable("bias",shape=(),dtype=tf.float32,
                    initializer=tf.zeros_initializer,
                    trainable=True)
y_hat = W * x_p + b
loss = tf.reduce_mean(tf.square(y_hat - y_p))

In [68]:
sess=tf.Session() # session of the default graph
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess.graph)
writer.flush()

In [69]:
gradient = tf.gradients(loss,[W,b])
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess.graph)
writer.flush()

In [70]:
sess.run(tf.global_variables_initializer())
W_init = sess.run([W,b])
print(W_init)
prediction = sess.run([y_hat,loss,gradient],feed_dict={x_p: x,y_p: y})
print(prediction)
for i in range(100):
    grad=sess.run(gradient,feed_dict={x_p: x,y_p: y})
    W_old,b_old = sess.run([W,b])
    sess.run([W.assign_sub(0.1*grad[0]),b.assign_sub(0.1*grad[1])]) #what about millions of weights?
#     sess.run([var.assign_sub(0.1*gradient) for var,gradient in zip(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES),grad)])
    W_new,b_new = sess.run([W,b])
    print(i,"gradient: ",grad, "\t", W_old,b_old, "==>",W_new,b_new)
print(sess.run([W,b]))

[1.0, 0.0]
[array([[1.],
       [2.],
       [3.],
       [4.]], dtype=float32), 5.0, [5.0, 0.0]]
0 gradient:  [5.0, 0.0] 	 1.0 0.0 ==> 0.5 0.0
1 gradient:  [-2.5, -2.5] 	 0.5 0.0 ==> 0.75 0.25
2 gradient:  [2.5, -0.75] 	 0.75 0.25 ==> 0.5 0.325
3 gradient:  [-0.87499976, -1.8499999] 	 0.5 0.325 ==> 0.5875 0.51
4 gradient:  [1.3624997, -1.0425001] 	 0.5875 0.51 ==> 0.45125002 0.61425
5 gradient:  [-0.15999997, -1.51525] 	 0.45125002 0.61425 ==> 0.46725002 0.76577497
6 gradient:  [0.8376254, -1.1321999] 	 0.46725002 0.76577497 ==> 0.38348746 0.87899494
7 gradient:  [0.14728665, -1.3245728] 	 0.38348746 0.87899494 ==> 0.3687588 1.0114522
8 gradient:  [0.5886431, -1.1333016] 	 0.3687588 1.0114522 ==> 0.3098945 1.1247823
9 gradient:  [0.2723291, -1.2009628] 	 0.3098945 1.1247823 ==> 0.2826616 1.2448786
10 gradient:  [0.46431732, -1.0969347] 	 0.2826616 1.2448786 ==> 0.23622985 1.354572
11 gradient:  [0.31630778, -1.1097066] 	 0.23622985 1.354572 ==> 0.20459908 1.4655427
12 gradient:  [0.39

94 gradient:  [0.029772878, -0.08753276] 	 -0.90055805 4.7076287 ==> -0.90353537 4.716382
95 gradient:  [0.028879642, -0.08491278] 	 -0.90353537 4.716382 ==> -0.90642333 4.7248735
96 gradient:  [0.02801776, -0.082369566] 	 -0.90642333 4.7248735 ==> -0.9092251 4.7331104
97 gradient:  [0.027175426, -0.079904675] 	 -0.9092251 4.7331104 ==> -0.91194266 4.741101
98 gradient:  [0.026363969, -0.07751179] 	 -0.91194266 4.741101 ==> -0.91457903 4.748852
99 gradient:  [0.025573254, -0.07519162] 	 -0.91457903 4.748852 ==> -0.9171364 4.756371
[-0.9171364, 4.756371]


In [29]:
prediction = sess.run([y_hat,loss,gradient],feed_dict={x_p: x,y_p: y})
print("prediction")
print(prediction[0])
print("target")
print(y)
# 1000 steps

prediction
[[3.8392346]
 [2.9220982]
 [2.004962 ]
 [1.0878255]]
target
[[4.]
 [3.]
 [2.]
 [1.]]


In [30]:
# single sigmoid neuron prediction on two-dimension data
x = np.array([[0,0],[1,0],[0,1],[1,1]])
y = np.array([[0],[0],[1],[1]])
#tf.sigmoid() tf.reduce_mean()

In [ ]:
#your implementation goes here, add more code cells if necessary

In [ ]:
# visualize the model
vis_inputs = np.random.uniform(size=(10000,2))
predictions = sess.run(y_hat,feed_dict={x_p:vis_inputs})
print(predictions.shape)
colors = list(map(lambda p: 'red' if p < 0.5 else 'blue', predictions))
plt.scatter(vis_inputs[:,0], vis_inputs[:,1], 1, colors)

In [ ]:
# single sigmoid neuron prediction on () two-dimension data
x = np.array([[0,0],[1,0],[0,1],[1,1]])
y = np.array([[0],[1],[1],[0]])

In [ ]:
# visualize the model
vis_inputs = np.random.uniform(size=(10000,2))
predictions = sess.run(y_hat,feed_dict={x_p:vis_inputs})
print(predictions.shape)
colors = list(map(lambda p: 'red' if p < 0.5 else 'blue', predictions))
plt.scatter(vis_inputs[:,0], vis_inputs[:,1], 1, colors)

# Summary
## 1. What is tensorflow?
## 2. Why do people use tensorflow?
## 3. How to write tensorflow program?
### (1). Build dataflow graph
### (2). Run dataflow graph and train the weights
## 4. (Bonus) What is feature learning and feature crafting (feature engineering), Difference, how to do feature crafting?

## 1. 

## 2.

## 3.(1). 

## 3.(2).

## 4.